# PYTORCH CNN Classifier

In [1]:
DATASET = 'none'
VOCAB_SIZE = 0
TOKENIZER = 'none'
KMER = 0

In [2]:
# Parameters
DATASET = "demo_coding_vs_intergenomic_seqs"
TOKENIZER = "character"


In [3]:
print(DATASET, VOCAB_SIZE, TOKENIZER, KMER)

demo_coding_vs_intergenomic_seqs 0 character 0


## Config

In [4]:
import torch
from torch.utils.data import DataLoader

from genomic_benchmarks.dataset_getters.pytorch_datasets import get_dataset
from glp.models import CNN
from glp.tokenizers import get_tokenizer
from glp.tokenizers.utils import build_vocab, coll_factory, check_config, check_seq_lengths

In [5]:
config = {
    "dataset": DATASET,
    "tokenizer": TOKENIZER,
    "dataset_version": 0,
    "epochs": 5,
    "batch_size": 32,
    "use_padding": True,
    "force_download": False,
    "run_on_gpu": True,
    "number_of_classes": 2,
    "embedding_dim": 100,
}
check_config(config)

## Choose the dataset

In [6]:
train_dset = get_dataset(config["dataset"], 'train')

## Tokenizer and vocab

In [7]:
tokenizer = get_tokenizer(config['tokenizer'])
tokenizer.train(train_dset=train_dset, vocab_size=VOCAB_SIZE, kmer=KMER)
vocabulary = build_vocab(train_dset, tokenizer, use_padding=config["use_padding"])

print("vocab len:" ,vocabulary.__len__())
print(vocabulary.get_stoi())

vocab len: 8
{'<eos>': 6, 'G': 2, '<pad>': 7, '<unk>': 0, '<bos>': 1, 'T': 4, 'C': 5, 'A': 3}


## Dataloader and batch preparation

In [8]:
# Run on GPU or CPU
device = 'cuda' if config["run_on_gpu"] and torch.cuda.is_available() else 'cpu'
print('Using {} device'.format(device))

max_tok_len = check_seq_lengths(dataset=train_dset, tokenizer=tokenizer)

# Data Loader
collate = coll_factory(vocabulary, tokenizer, device, pad_to_length = max_tok_len)

train_loader = DataLoader(train_dset, batch_size=config["batch_size"], shuffle=True, collate_fn=collate)

Using cpu device


max_tok_len  202


In [9]:
tokenizer(train_dset[1][0])

['<bos>',
 'C',
 'C',
 'A',
 'T',
 'C',
 'C',
 'G',
 'A',
 'G',
 'T',
 'G',
 'T',
 'C',
 'C',
 'T',
 'A',
 'G',
 'T',
 'G',
 'A',
 'A',
 'T',
 'C',
 'T',
 'G',
 'C',
 'A',
 'G',
 'C',
 'C',
 'C',
 'A',
 'G',
 'A',
 'T',
 'G',
 'G',
 'G',
 'C',
 'T',
 'C',
 'A',
 'C',
 'G',
 'T',
 'T',
 'T',
 'C',
 'C',
 'C',
 'T',
 'G',
 'T',
 'G',
 'T',
 'A',
 'C',
 'C',
 'C',
 'C',
 'A',
 'C',
 'A',
 'T',
 'G',
 'C',
 'C',
 'C',
 'A',
 'C',
 'T',
 'G',
 'A',
 'G',
 'C',
 'C',
 'C',
 'C',
 'G',
 'A',
 'A',
 'G',
 'T',
 'G',
 'G',
 'T',
 'T',
 'G',
 'C',
 'C',
 'C',
 'A',
 'G',
 'A',
 'C',
 'C',
 'C',
 'C',
 'T',
 'T',
 'A',
 'C',
 'C',
 'C',
 'A',
 'G',
 'A',
 'C',
 'C',
 'C',
 'C',
 'G',
 'G',
 'G',
 'C',
 'T',
 'G',
 'A',
 'A',
 'A',
 'G',
 'A',
 'A',
 'C',
 'C',
 'T',
 'T',
 'G',
 'T',
 'G',
 'T',
 'G',
 'G',
 'T',
 'T',
 'T',
 'A',
 'A',
 'T',
 'C',
 'T',
 'C',
 'A',
 'G',
 'A',
 'G',
 'C',
 'A',
 'G',
 'A',
 'G',
 'T',
 'C',
 'C',
 'T',
 'C',
 'T',
 'G',
 'A',
 'G',
 'C',
 'T',
 'G',
 'G',
 'A',


## Model

In [10]:
model = CNN(
    number_of_classes=config["number_of_classes"],
    vocab_size=vocabulary.__len__(),
    embedding_dim=config["embedding_dim"],
    input_len=max_tok_len
).to(device)

## Training

In [11]:
model.train(train_loader, epochs=config["epochs"])

Epoch 0


/Users/katarina/Documents/repositories/GitHub/GLP/src/glp/tokenizers/utils.py:27: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  x = torch.tensor(pad(x), dtype=torch.long)


Train metrics: 
 Accuracy: 85.1%, Avg loss: 0.570226 

Epoch 1


Train metrics: 
 Accuracy: 86.0%, Avg loss: 0.566634 

Epoch 2


Train metrics: 
 Accuracy: 86.4%, Avg loss: 0.564500 

Epoch 3


Train metrics: 
 Accuracy: 86.0%, Avg loss: 0.564757 

Epoch 4


Train metrics: 
 Accuracy: 87.0%, Avg loss: 0.562938 



## Testing

In [12]:
test_dset = get_dataset(config["dataset"], 'test')
test_loader = DataLoader(test_dset, batch_size=config["batch_size"], shuffle=True, collate_fn=collate)

acc, f1 = model.test(test_loader)
acc, f1

p  12500 ; tp  10339.36441707611 ; fp  1239.818584475852
recall  0.8271491533660889 ; precision  0.892926937564621
num_batches 782
correct 21634
size 25000
Test metrics: 
 Accuracy: 0.865360, F1 score: 0.858780, Avg loss: 0.565353 



(0.86536, 0.8587803345661449)